<a href="https://colab.research.google.com/github/karino2/tegashiki/blob/master/tegashiki_vec_tputrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tegashiki**

In [1]:
# today tensorflow seems update and batchnormalization name scope is changed. I downgrade for a while
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 1.3MB/s 
     |████████████████████████████████| 368kB 37.1MB/s 
  Found existing installation: tensorflow-estimator 1.14.0rc1
    Uninstalling tensorflow-estimator-1.14.0rc1:
      Successfully uninstalled tensorflow-estimator-1.14.0rc1
  Found existing installation: tensorflow 1.14.0rc1
    Uninstalling tensorflow-1.14.0rc1:
      Successfully uninstalled tensorflow-1.14.0rc1


In [0]:
import os
import datetime
import pickle
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing

In [3]:
tf.VERSION

'1.13.1'

In [4]:
from google.colab import auth
auth.authenticate_user()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
def get_if_necessary(fname):
  if os.path.exists(fname):
    return
  !gsutil cp gs://karino2-tegashiki/dataset/{fname} {fname}

def get_and_load(fname):
  get_if_necessary(fname)
  with gzip.open(fname,'rb') as f:
    return pickle.load(f)  
  
def send_file(fname):
  !gsutil cp {fname} gs://karino2-tegashiki/dataset/
    
def dump_and_send(obj, fname):
  with gzip.open(fname,'wb') as f:
    pickle.dump(obj, f)
  send_file(fname)


In [0]:
BEGIN_OF_SEQ = 112
END_OF_SEQ=113
PAD_TOKEN=0


# Both train and valid
# MAX_STROKE_NUM=115
MAX_ONE_STROKE_LEN=50
MAX_STROKE_NUM=13

# MAX_TEX_LEN=206+2
# +2 is bos, eos
# +2 is bos, eos
# MAX_TOKEN_LEN=88+2
# MAX_TRAIN_LEN=88+2
MAX_TOKEN_LEN=3+2

VOCAB_SIZE=114


NORMALIZE_MAX=2000

# must match to trained dim
EXTRACTED_FEATURE_DIM=256
FE_DROPOUT_RATE=0.5
FE_L2_REGULARIZATION_RATE=0.1

INPUT_TYPE_POINT=1
INPUT_TYPE_END=0

# (x, y, TYPE)
INPUT_TYPE_DIM=3

### Model

In [0]:
import tensorflow as tf

from tensorflow.keras.layers import Input, GRU, Dense, RepeatVector, Reshape, Concatenate
from tensorflow.keras.layers import TimeDistributed, Flatten, Lambda, Add, Activation, Masking, Embedding
from tensorflow.keras.layers import AveragePooling1D, Conv1D, MaxPooling1D, SpatialDropout1D
from tensorflow.keras.layers import BatchNormalization, Dropout, GlobalMaxPooling1D
from tensorflow.keras import regularizers
import  tensorflow.keras.layers as layers
# from tensorflow.keras.layers.embeddings import Embedding
import tensorflow.keras.backend as K

In [0]:
#DROPOUT_RATE=0.9
DROPOUT_RATE=0.5
L2_REGULARIZATION_RATE=0.1

FEATURE_EXTRACTER_KERNEL_SIZE=7

# FILTER_SIZE=3
# KERNEL_SIZE=8
FILTER_SIZE=5
KERNEL_SIZE=128


# large

# EMBEDDING_SIZE=256
# OT_HIDDEN=256
# GRU_HIDDEN=256
# ATTENTION_ENC_HIDDEN=256
# ATTENTION_DEC_HIDDEN=256

# model_small
EMBEDDING_SIZE=32
OT_HIDDEN=128
GRU_HIDDEN=128
ATTENTION_ENC_HIDDEN=64
ATTENTION_DEC_HIDDEN=64

# model_small_embed acc 0.22
# EMBEDDING_SIZE=32

# OT_HIDDEN=256
# GRU_HIDDEN=256
# ATTENTION_ENC_HIDDEN=256
# ATTENTION_DEC_HIDDEN=256



In [0]:
def feature_extractor(input_stroke_t, is_training):
  """input_stroke_t shape (batch, MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM)
     output: (batch, MAX_STROKE_NUM, EXTRACTED_FEATURE_DIM)"""
  with tf.variable_scope("feature_extractor"):
    inpshape = input_stroke_t.shape
    x = tf.reshape(input_stroke_t, [-1, inpshape[2], inpshape[3]])
    # (batch*MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM)

    x = Conv1D(32, FEATURE_EXTRACTER_KERNEL_SIZE, kernel_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE))(x)
    x = BatchNormalization()(x, training=is_training)
    x = Activation('relu')(x)
    # (batch*MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, 32)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(FE_DROPOUT_RATE)(x, training=is_training)
    # (batch*MAX_STROKE_NUM, MAX_ONE_STROKE_LEN/2, 32)

    x = Conv1D(64, FEATURE_EXTRACTER_KERNEL_SIZE, kernel_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE))(x)
    x = BatchNormalization()(x, training=is_training)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(FE_DROPOUT_RATE)(x, training=is_training)
    # (batch*MAX_STROKE_NUM, MAX_ONE_STROKE_LEN/4, 64)

    x = Conv1D(EXTRACTED_FEATURE_DIM, 7, kernel_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(FE_L2_REGULARIZATION_RATE))(x)
    x = BatchNormalization()(x, training=is_training)
    x = Activation('relu')(x)
    x = Dropout(FE_DROPOUT_RATE)(x, training=is_training)
    x = GlobalMaxPooling1D()(x)
    x = tf.reshape(x, [-1, inpshape[1], EXTRACTED_FEATURE_DIM])
    return x
  
def attention_context(ht_enc, ht_dec, maxtklen):
  w1 = Dense(ATTENTION_ENC_HIDDEN, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(ht_enc)
  w2 = Dense(ATTENTION_DEC_HIDDEN, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(ht_dec)
  # w1 (sample, 276, 256)
  # w2 (samples, 90, 256)


  w2_widen = tf.expand_dims(w2, axis=1)
  # (sample, 1, 90, 256)

  w1_widen = tf.expand_dims(w1, axis=2)
  # (sample, 276, 1, 256)

  w1_widen_repeat = K.repeat_elements(w1_widen, rep=maxtklen, axis=2)
  # (sample, 276, 90, 256)
  
  score =tf.nn.tanh(w1_widen_repeat+w2_widen)
  prob = Dense(1, activation="softmax", kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(score)
  # score: (sample, 276, 90, 256)
  # prob: (sample, 276, 90, 1)

  ht_enc_repeated = K.repeat_elements(tf.expand_dims(ht_enc, axis=2), rep=maxtklen, axis=2)
  # (sample, 276, 90, 256)

  context_vec = tf.reduce_sum(prob*ht_enc_repeated, axis=1)
  # (sample, 90, 256)

  return context_vec  

In [0]:
def encoder_CNN(stroke_enc, is_training, dropout_rate=DROPOUT_RATE):
  x = Conv1D(64, 8, activation='relu', padding='same', kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(stroke_enc)
  x = tf.contrib.layers.layer_norm(x)
  x = SpatialDropout1D(DROPOUT_RATE)(x, training=is_training)
  x = Conv1D(32, 8, activation='relu', padding='same',kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(x)
  x = tf.contrib.layers.layer_norm(x)
  x = SpatialDropout1D(DROPOUT_RATE)(x, training=is_training)
  return x


In [0]:
"""
def myembedding(input, num_classes, embedding_size, name):
  randinitializer = lambda: tf.random_uniform([num_classes, embedding_size], -1.0, 1.0)
  embedmat = tf.get_variable(name, initializer = randinitializer)
  return tf.nn.embedding_lookup(embedmat, input)  
"""

# dynamic shape cause TPUEstimator export to fail...
def myembedding(input, num_classes, embedding_size, seq_num, name):
  with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
    randinitializer = lambda: tf.random_uniform([num_classes, embedding_size], -1.0, 1.0)

    embedmat = tf.get_variable(name, initializer = randinitializer)
    # embedding_lookup with generated tensor (eg. tf.range) cause TFLite convert fail.
    # return tf.nn.embedding_lookup(embedmat, input)
    onehot = tf.one_hot(input, num_classes)
    # batch_num, seqnum = onehot.shape[0], onehot.shape[1]    
    # flatten_onehot = tf.reshape(onehot, [batch_num*seq_num, num_classes])
    flatten_onehot = tf.reshape(onehot, [-1, num_classes])
    return tf.reshape(tf.matmul(flatten_onehot, embedmat), [-1, seq_num, embedding_size])


In [0]:


#input_from_encoder_t (8190, 2773, 3)
#decoder_input_t (batch, tklen, 3)

def create_model(input_stroke_t, decoder_input_t, is_training):
  stroke_features = feature_extractor(input_stroke_t, is_training)
  # (batch, MAX_STROKE_NUM, EXTRACTED_FEATURE_DIM)
  
  # ht_enc, state_enc = encoder_TCN(input_from_encoder_t)
  # ht_enc, state_enc = encoder_CNNRNN(input_from_encoder_t)
  # ht_enc, state_enc = GRU(GRU_HIDDEN, return_sequences=True,return_state=True, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), recurrent_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(stroke_features)

  pos_stroke = tf.range(
            0,
            tf.shape(stroke_features)[1],
            delta=1,
            dtype=tf.int32,
            name='range')
  # pos_stroke = tf.expand_dims(tf.expand_dims(pos_stroke, axis=1), axis=0)
  pos_stroke = tf.expand_dims(pos_stroke, axis=0)
  pos_stroke_embed = myembedding(pos_stroke, MAX_STROKE_NUM, EXTRACTED_FEATURE_DIM, MAX_STROKE_NUM, "stroke_pos_embed")
  # pos_stroke_embed = Embedding(MAX_STROKE_NUM, EXTRACTED_FEATURE_DIM, input_length=MAX_STROKE_NUM)(pos_stroke)
  
  stroke_pos_embedded = stroke_features + tf.cast(x=pos_stroke_embed, dtype=stroke_features.dtype)
  
  # InvalidArgumentError: Dimensions must be equal, but are 32 and 13 for 'add_1' (op: 'Add')
  #   with input shapes: [32,13,256], [1,13,1,256].


  # ht_enc = stroke_pos_embedded
  ht_enc = encoder_CNN(stroke_pos_embedded, is_training)
 
  # dec_input_embedded = Embedding(VOCAB_SIZE, EMBEDDING_SIZE, input_length=decoder_input_t.shape[1])(decoder_input_t)
  dec_input_embedded = myembedding(decoder_input_t, VOCAB_SIZE, EMBEDDING_SIZE, MAX_TOKEN_LEN, "dec_embed")
  # (batch, 98, 256)
  
  dec_pos_input = tf.range(
            0,
            tf.shape(decoder_input_t)[1],
            delta=1,
            dtype=tf.int32,
            name='range')
  
  # [5] -> [1, 5, 1]
  # dec_pos_input = tf.expand_dims(tf.expand_dims(dec_pos_input, axis=1), axis=0)
  # [5] -> [1, 5]
  dec_pos_input = tf.expand_dims(dec_pos_input, axis=0)
  # dec_pos_embed = Embedding(MAX_TOKEN_LEN, EMBEDDING_SIZE, input_length=MAX_TOKEN_LEN)(dec_pos_input)
  dec_pos_embed = myembedding(dec_pos_input, MAX_TOKEN_LEN, EMBEDDING_SIZE, MAX_TOKEN_LEN, "dec_pos_embed")
  
  
  dec_embedded = dec_input_embedded + tf.cast(x=dec_pos_embed, dtype=dec_input_embedded.dtype)
  # Dimensions must be equal, but are 32 and 5 for 'add_2' (op: 'Add') with input shapes:
  # [32,5,32], [1,5,1,32].
  
  x = Conv1D(FILTER_SIZE, KERNEL_SIZE, activation='relu', padding='causal', kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(dec_embedded)
  x = tf.contrib.layers.layer_norm(x)
  ht_dec = SpatialDropout1D(DROPOUT_RATE)(x, training=is_training)


  context_vec = attention_context(ht_enc, ht_dec, decoder_input_t.shape[1])
  # context_vec = attention_context(conved, ht_dec, maxtklen)
  # (sample, 90, 256)

  # ht: (sample, 98, 256)
  # ot_input: (sample, 98, 1536) 

  ot_input = Concatenate()([ht_dec, context_vec])

  # (Sample, 98, 256)
  ot = Dense(OT_HIDDEN, activation="tanh", kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(ot_input)

  # (Sample, 98, 112)
  logit = TimeDistributed(Dense(VOCAB_SIZE, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE)))(ot)

  return logit


In [0]:
def sparse_softmax_cross_entropy_with_mask(sparse_labels, logit, mask):
  mask = tf.cast(mask, tf.float32)
  mask_expands = tf.expand_dims(mask, axis=2)
  return tf.losses.sparse_softmax_cross_entropy(sparse_labels, logit, mask_expands)

In [0]:
from tqdm.autonotebook import tqdm as tqdmn

## TPUEstimator

In [0]:
DATA_DIR="gs://karino2-tegashiki/dataset"
TF_RECORD_FILE="{}/crohme2019_onesupsubtwo_train.tfrecord.gz".format(DATA_DIR)
TF_VALID_RECORD_FILE="{}/crohme2019_onesupsubtwo_valid.tfrecord.gz".format(DATA_DIR)
FEATURE_EXTRACTOR_DIR="gs://karino2-tegashiki/sym_models/rnn_fdim256"

In [0]:
# MODEL_DIR="gs://karino2-tegashiki/models/padstroke_small_rnn_small_dropout05"
# MODEL_DIR="gs://karino2-tegashiki/models/samelen_small_rnn_small_dropout05"
# MODEL_DIR="gs://karino2-tegashiki/models/samelen_nostroke"
# MODEL_DIR="gs://karino2-tegashiki/models/varlen_10_small_rnn_small_dropout05"

# MODEL_DIR="gs://karino2-tegashiki/models/expgen_rnn_small_dropout05"
# MODEL_DIR="gs://karino2-tegashiki/models/expgen_litelstm"
# MODEL_DIR="gs://karino2-tegashiki/models/expgen_convdec"

# MODEL_DIR="gs://karino2-tegashiki/models/convdec_f5k128"
# MODEL_DIR="gs://karino2-tegashiki/models/convdec_f5k128_storkeposenc"

# MODEL_DIR="gs://karino2-tegashiki/models/convencdec"
# MODEL_DIR="gs://karino2-tegashiki/models/convencdec_posfc"

# MODEL_DIR="gs://karino2-tegashiki/models/convencdec_myembed"
MODEL_DIR="gs://karino2-tegashiki/models/convencdec_myembed2"



In [132]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.26.129.210:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1418945649334182195),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15885677594514027856),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9298473833439319927),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7117834393050913660),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3230998589906857640),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 11267132420472778475),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 8335039842009929798),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 8662724353136131612),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 6337943953445

In [0]:
VALID_SAMPLE_NUM=23884

In [0]:
TRAIN_STEP_PER_ONCE=1000
EVAL_BATCH_SIZE=8

In [0]:
def parser(serialized_example):
  featurelen = MAX_STROKE_NUM*MAX_ONE_STROKE_LEN
  
  features = tf.parse_single_example(
      serialized_example,
      features={
      'input_x': tf.FixedLenFeature([featurelen], tf.int64),
      'input_y': tf.FixedLenFeature([featurelen], tf.int64),
      'input_type': tf.FixedLenFeature([featurelen], tf.int64),
      'decoder_input':tf.FixedLenFeature([MAX_TOKEN_LEN], tf.int64),
      'decoder_labels':tf.FixedLenFeature([MAX_TOKEN_LEN], tf.int64)}          
  )
  
  input_x, input_y, input_type = [tf.reshape(tf.cast(features[fname], tf.int32), [MAX_STROKE_NUM, MAX_ONE_STROKE_LEN]) for fname in ['input_x', 'input_y', 'input_type']]
  one_sample_stroke = tf.stack([input_x, input_y, input_type], 2)
  decoder_input = tf.cast(features["decoder_input"], tf.int32)
  decoder_labels = tf.cast(features["decoder_labels"], tf.int32)
  
  return {"input_stroke": one_sample_stroke, "input_decoder": decoder_input} , decoder_labels

In [0]:
def tpu_input_fn(params):
  dataset = tf.data.TFRecordDataset(TF_RECORD_FILE, "GZIP")
  dataset = dataset.map(parser)
  dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.batch(params['batch_size'], drop_remainder=True)
  return dataset

def tpu_input_fn_valid(params):
  dataset = tf.data.TFRecordDataset(TF_VALID_RECORD_FILE, "GZIP")
  dataset = dataset.map(parser)
  dataset = dataset.batch(params['batch_size'], drop_remainder=True)
  return dataset

In [0]:
def metric_fn(labels, logits, predicted_classes, mask_weights):
    """Function to return metrics for evaluation."""

      
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   weights=mask_weights,
                                   name="acc_op")
    return {"accuracy": accuracy}

def extract_params(features, mode, params): 
  input_stroke = tf.feature_column.input_layer(features, params['input_stroke'])

  #[MAX_STROKE_NUM, MAX_ONE_STROKE_LEN]
  input_stroke = tf.reshape(input_stroke, shape=(-1,MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM))
  input_for_dec= tf.feature_column.input_layer(features, params['input_for_dec'])
  # Why I have to?
  input_for_dec = tf.cast(input_for_dec,tf.int32)
  
  return (input_stroke, input_for_dec)
  
def tpu_model_fn(features, labels, mode, params):
  input_stroke, input_for_dec = extract_params(features, mode, params)
  # input_stroke, input_for_dec, maxtklen = extract_params_always_train(features, mode, params)
  
  logit = create_model(input_stroke, input_for_dec, mode==tf.estimator.ModeKeys.TRAIN)
  # maxtklen = MAX_TOKEN_LEN
  # logit = create_model_nostroke(input_stroke, input_for_dec, maxtklen)
  
  
  mask = tf.not_equal(input_for_dec, 0)
  mask_int = tf.cast(mask, tf.int64)
  
  predicted_classes = tf.math.argmax(logit,axis=2)*mask_int
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {
        "class_ids": predicted_classes[:, tf.newaxis],
        "logits": logit,
    }
    return tf.contrib.tpu.TPUEstimatorSpec(mode, predictions=predictions)  
  
  loss = sparse_softmax_cross_entropy_with_mask(labels, logit, mask)

  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, loss=loss, eval_metrics=(metric_fn, [labels, logit, predicted_classes, tf.cast(mask, tf.float32)]))
  
  optimizer = tf.train.AdamOptimizer(params['learning_rate'])  
  optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)  
  
  train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
  return tf.contrib.tpu.TPUEstimatorSpec(mode, loss=loss, train_op=train_op)


In [0]:
def get_global_step(estimator):
  try:
    return int(estimator.get_variable_value("global_step"))
  except ValueError:
    return 0
      
def train_tpu_estimator(tpu_estimator, max_steps):
  step = get_global_step(tpu_estimator)+TRAIN_STEP_PER_ONCE
  while step < max_steps:
    tpu_estimator.train(
      input_fn = tpu_input_fn,
      max_steps=step)
    eval_results = tpu_estimator.evaluate(
      input_fn=tpu_input_fn_valid,
      steps= VALID_SAMPLE_NUM// EVAL_BATCH_SIZE)
    print(step)
    print(eval_results)
    step += TRAIN_STEP_PER_ONCE
  tpu_estimator.train(
    input_fn = tpu_input_fn,
    max_steps=max_steps)
  eval_results = tpu_estimator.evaluate(
    input_fn=tpu_input_fn_valid,
    steps= VALID_SAMPLE_NUM// EVAL_BATCH_SIZE)


In [0]:
# !gsutil ls gs://karino2-tegashiki/sym_models/rnn_fdim256

In [0]:
wss = tf.estimator.WarmStartSettings(ckpt_to_initialize_from=FEATURE_EXTRACTOR_DIR, vars_to_warm_start=".*feature_extractor.*")
# wss = None

In [0]:
cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2


### TPUEstimator instantiation

In [181]:
run_config = tf.contrib.tpu.RunConfig(
    cluster=cluster_resolver,
    master=None,
    model_dir=MODEL_DIR,
    save_checkpoints_steps=100,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=1000,
        num_shards=8,
        per_host_input_for_training=is_per_host
        # per_host_input_for_training=False
    ))

tpu_estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=tpu_model_fn,
    config=run_config,
    export_to_tpu=False, # Conv1D cause error for TPU graph with ReadVariableOp. why?
    params={
        'learning_rate': 0.00009,
#        'learning_rate': 0.001,
        'input_stroke':tf.feature_column.numeric_column(key="input_stroke", shape=(MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM)),
        'input_for_dec': tf.feature_column.numeric_column(key="input_decoder", shape=(MAX_TOKEN_LEN,), dtype=tf.int32),
    },
    warm_start_from=wss,
    train_batch_size=8*32,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=8)

INFO:tensorflow:Using config: {'_model_dir': 'gs://karino2-tegashiki/models/convencdec_myembed2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.26.129.210:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc27f519518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.26.129.210:8470', '_evaluation_master': 'grpc://10.26.129.210:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shard

### TPUEstimator train

In [182]:
train_tpu_estimator(tpu_estimator, 2000)

INFO:tensorflow:Querying Tensorflow master (grpc://10.26.129.210:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 1418945649334182195)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15885677594514027856)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9298473833439319927)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7117834393050913660)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3230998589906857640)
INFO:tensorflow:*** Available Device: _DeviceAttributes(

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
train_tpu_estimator(tpu_estimator, 200000)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


143000
{'accuracy': 0.8623492, 'loss': 0.45740882, 'global_step': 143000}
144000
{'accuracy': 0.85991895, 'loss': 0.46983764, 'global_step': 144000}
145000
{'accuracy': 0.8595844, 'loss': 0.47163793, 'global_step': 145000}
146000
{'accuracy': 0.8663144, 'loss': 0.4473205, 'global_step': 146000}
147000
{'accuracy': 0.86489755, 'loss': 0.44887066, 'global_step': 147000}
148000
{'accuracy': 0.8566523, 'loss': 0.48246366, 'global_step': 148000}
149000
{'accuracy': 0.8541925, 'loss': 0.49169132, 'global_step': 149000}
150000
{'accuracy': 0.8602731, 'loss': 0.46691585, 'global_step': 150000}
151000
{'accuracy': 0.8532283, 'loss': 0.4923954, 'global_step': 151000}
152000
{'accuracy': 0.85929906, 'loss': 0.47052586, 'global_step': 152000}
153000
{'accuracy': 0.858935, 'loss': 0.47593313, 'global_step': 153000}
154000
{'accuracy': 0.86124724, 'loss': 0.46408913, 'global_step': 154000}
155000
{'accuracy': 0.86152273, 'loss': 0.46337906, 'global_step': 155000}
156000
{'accuracy': 0.85716397, 'los

KeyboardInterrupt: ignored

In [0]:
train_tpu_estimator(tpu_estimator, 120100)

### TensorBoard

In [0]:
!git clone https://github.com/mixuala/colab_utils

Cloning into 'colab_utils'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 65.93 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [0]:
kill_tensorboard()

IndexError: ignored

In [0]:
import os
import colab_utils.tboard

ROOT = %pwd
colab_utils.tboard.launch_tensorboard(bin_dir=ROOT, log_dir=MODEL_DIR)
print(MODEL_DIR)

calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ...
calling unzip ngrok-stable-linux-amd64.zip ...
ngrok installed. path=/content/ngrok
status: tensorboard=False, ngrok=False
tensorboard url= https://056205b1.ngrok.io
gs://karino2-tegashiki/models/convencdec_posfc


In [0]:
import re

def find_one_command(res_arr, word):
  return list(filter(lambda arr: arr[4] == word, res_arr))[0]

def kill_tensorboard():
  ps_res = !ps
  res_arr = [re.split(r' +', one) for one in ps_res[1:]]
  # pid_ngrok = find_one_command(res_arr, "ngrok")[1]
  pid_tb = find_one_command(res_arr, "tensorboard")[1]
  # !kill {pid_ngrok}
  !kill {pid_tb}

In [0]:
# kill_tensorboard()

In [0]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
     10 ?        00:00:14 node
     25 ?        00:00:20 jupyter-noteboo
    120 ?        00:00:00 tail
   3260 ?        00:00:05 python3
   3330 ?        00:00:00 python3
   3352 ?        00:00:00 ps


### Retry procedure

In [0]:
# MODEL_DIR="gs://karino2-tegashiki/models/cnnrnn_small_dropout05"
MODEL_DIR="gs://karino2-tegashiki/models/cnnrnn_small_dropout09"

In [0]:
run_config = tf.contrib.tpu.RunConfig(
    cluster=cluster_resolver,
    master=None,
    model_dir=MODEL_DIR,
    save_checkpoints_steps=100,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=1000,
        num_shards=8,
        per_host_input_for_training=is_per_host
        # per_host_input_for_training=False
    ))

tpu_estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=tpu_model_fn,
    config=run_config,
    export_to_tpu=False, # Conv1D cause error for TPU graph with ReadVariableOp. why?
    params={
        'learning_rate': 0.00009,
#        'learning_rate': 0.001,
        'input_stroke':tf.feature_column.numeric_column(key="input_stroke", shape=(MAX_STROKE_SEQ_LEN, INPUT_TYPE_DIM)),
        'input_for_dec': tf.feature_column.numeric_column(key="input_decoder", shape=(MAX_TRAIN_LEN,)),
        'input_stroke_valid':tf.feature_column.numeric_column(key="input_stroke", shape=(MAX_VALID_STROKE_LEN, INPUT_TYPE_DIM)),
        'input_for_dec_valid': tf.feature_column.numeric_column(key="input_decoder", shape=(MAX_VALID_LEN,))
    },
    train_batch_size=8*32,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=8)

INFO:tensorflow:Using config: {'_model_dir': 'gs://karino2-tegashiki/models/cnnrnn_small_dropout09', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.66.74.90:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc90569e748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.66.74.90:8470', '_evaluation_master': 'grpc://10.66.74.90:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8

In [0]:
kill_tensorboard()

In [0]:
colab_utils.tboard.launch_tensorboard(bin_dir=ROOT, log_dir=MODEL_DIR)
print(MODEL_DIR)

ngrok installed
status: tensorboard=False, ngrok=True
tensorboard url= https://945ce833.ngrok.io
gs://karino2-tegashiki/models/cnnrnn_small_dropout09


In [0]:
train_tpu_estimator(40000)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://karino2-tegashiki/models/cnnrnn_small_dropout09/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20000 into gs://karino2-tegashiki/models/cnnrnn_small_dropout09/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 73 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (1000) batch(es) of data to infeed.
IN

# Model Export

Currently, just use the same limitation of input shape for training time to confirm conversion, etc.

In [0]:
tf.logging.set_verbosity(tf.logging.WARN)

In [0]:
MAX_STROKE_NUM*MAX_ONE_STROKE_LEN*INPUT_TYPE_DIM

1950

In [0]:
MAX_STROKE_NUM,MAX_ONE_STROKE_LEN,INPUT_TYPE_DIM

(13, 50, 3)

In [0]:
def create_serving_input():
  stroke_input_t = tf.placeholder(tf.int32, shape=[1, MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM], name='stroke_input')
  decoder_input_t = tf.placeholder(tf.int32, shape=[1, MAX_TOKEN_LEN], name='input_decoder')

  input_fn_recv = tf.estimator.export.build_raw_serving_input_receiver_fn({
      "input_stroke": stroke_input_t,
      "input_decoder": decoder_input_t
  })
  return input_fn_recv

In [0]:
# EXPORT_DIR="gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape"
# EXPORT_DIR="gs://karino2-tegashiki/models/exports/export_expgen_convencdec_posfc"
# EXPORT_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed"
EXPORT_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed2"


In [183]:
  tpu_estimator.export_savedmodel(
    export_dir_base=EXPORT_DIR,
    serving_input_receiver_fn=create_serving_input())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from gs://karino2-tegashiki/models/convencdec_myembed2/model.ckpt-2000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed2/temp-b'1560910035'/saved_model.pb


b'gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed2/1560910035'

# Misc

In [0]:
import math

In [0]:
# 1+2*(k-1)*(2^n-1) #> 500
# 1+2*7*(2^n-1) > 500
# 2^n-1 > 499/14
# 2^n > 1+499/14
math.log2(1+ (499/14.0))

In [0]:
# 1+2*(k-1)*(2^n-1) #> 500
# 1+2*7*(2^n-1) > 500
# 2^n-1 > 499/14
# 2^n > 1+499/14
math.log2(1+ (499/14.0))

 https://arxiv.org/abs/1803.01271

try to cover 500 len. and attention will handle larger case.
As paper noted, best kernel size depend on task.
I start from k=8 because our task is somewhat similar to P-MNIST, and k=8 is best for that task.

In [0]:
# TCN residual block in paper
# filter_size must be the same as out_channels?

# filter_size=32
def TCNResBlock(input, layer_depth, filter_size=3, kernel_size=8, dropout_rate=0.2):
  d = 2**layer_depth
  x = Conv1D(filter_size, kernel_size, activation='relu', dilation_rate=d, padding='causal', kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(input)
  
  # https://github.com/ychfan/tf_estimator_barebone/blob/master/common/layers.py
  # weight norm implementation. But I use layer_norm for first trial.
  x = tf.contrib.layers.layer_norm(x)
  x = SpatialDropout1D(dropout_rate)(x)

  x = Conv1D(filter_size, kernel_size, activation='relu', dilation_rate=d, padding='causal', kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(x)
  x = tf.contrib.layers.layer_norm(x)
  x = SpatialDropout1D(dropout_rate)(x)
  return tf.nn.relu(x + input)

def TCN(input, depth=8):
  x = input
  for i in range(depth):
    x = TCNResBlock(x, i)
  return x


In [0]:
def encoder_TCN(input_from_encoder_t):
  conved = TCN(input_from_encoder_t)

  state_for_dec = Dense(GRU_HIDDEN, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(conved[:, -1, :])
  
  pooled = AveragePooling1D(10)(conved)
  return pooled, state_for_dec

def encoder_CNNRNN(input_from_encoder_t, dropout_rate=DROPOUT_RATE):
  conved = Conv1D(32, 7, activation='relu', kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(input_from_encoder_t)
  pooled = AveragePooling1D(10)(conved)

  ht_enc, state_enc = GRU(GRU_HIDDEN, return_sequences=True,return_state=True, dropout=dropout_rate, recurrent_dropout=dropout_rate, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), recurrent_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(pooled)
  return ht_enc, state_enc

In [0]:
def create_model_nostroke(input_stroke_t, decoder_input_t, maxtklen=MAX_TOKEN_LEN):
  dec_input_embedded = Embedding(VOCAB_SIZE, EMBEDDING_SIZE, input_length=maxtklen)(decoder_input_t)
  # (batch, 98, 256)


  # (sample, timestamps, htdim)
  ht_last = GRU(GRU_HIDDEN, return_sequences=True, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), recurrent_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(dec_input_embedded)

  # (Sample, 98, 112)
  logit = TimeDistributed(Dense(VOCAB_SIZE, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE)))(ht_last)

  return logit
